In [ ]:
import numpy as np
from collections import deque
import random

import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.autograd
from torch.autograd import Variable

# from enviroment import Env_cellular as env
import matplotlib.pyplot as plt

In [ ]:
import scipy
from scipy import special
import numpy as np
from scipy.special import lambertw
import math

dtype = np.float32

In [ ]:
##################################################################################################################
# Ornstein-Ulhenbeck Process
# Put some noise into policy to make new exploration outside the right policy (exploitation)
# Promote exploration -> Gaussian noise is added to the action determined by the policy
class OUNoise(object):
  def __init__(self, action_space, mu=0.0, theta=0.15, max_sigma=0.3, min_sigma=0.3, decay_period=100000):
    self.mu           = mu                      # mu: policy function 
    self.theta        = theta                   # theta: parameters 
    self.sigma        = max_sigma               # initial value of sigma is at maximum value of sigma
    self.max_sigma    = max_sigma               # 
    self.min_sigma    = min_sigma               # 
    self.decay_period = decay_period            # decay_period:
    self.action_dim   = action_space.shape[0]   # action_dim: dimension of the action space
    self.low          = action_space.low        # low: 
    self.high         = action_space.high       # high:
    self.reset()                                # reset:

  # Reset the state matrix back to matrix of MU <policy>
  def reset(self):
    self.state = np.ones(self.action_dim) * self.mu

  # 
  def evolve_state(self):
    # buffer the state to x
    x  = self.state
    dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(self.action_dim)

    # state update
    self.state = x + dx
    return self.state

  def get_action(self, action, t=0):
    ou_state = self.evolve_state() # new state <Ornstein-Uhlenbeck state>
    self.sigma = self.max_sigma - (self.max_sigma - self.min_sigma) * min(1.0, t / self.decay_period)
    return np.clip(action + ou_state, self.low, self.high) 
        
##################################################################################################################
# Work similar to what replay buffer 's theory do:

class Memory:
  def __init__(self, max_size):
    self.max_size = max_size # max_size of deque is number of experience that allows to be saved in deque
    self.buffer = deque(maxlen=max_size) # deque||double-end queue has the feature of adding and removing elements from either end || similar to numpy array but easier to work with
  
  # push new Experience into buffer // push old Experience out of buffer
  def push(self, state, action, reward, next_state, done): 
    experience = (state, action, np.array([reward]), next_state, done) # why reward is in array 
    self.buffer.append(experience) # append new experience//observation into deque. If the data reach the buffer 's capacity limit => delete oldest data

  def sample(self, batch_size):
    state_batch = []
    action_batch = []
    reward_batch = []
    next_state_batch = []
    done_batch = []

    batch = random.sample(self.buffer, batch_size)  # pick random batch (a packet of random data with size = batch_size // from buffer)

    for experience in batch:  # loops over all group of [state, action, reward, next_state, done] in batch
      state, action, reward, next_state, done = experience
      # append all [state, action, reward, next_state, done] into batch // split into separated features
      state_batch.append(state)
      action_batch.append(action)
      reward_batch.append(reward)
      next_state_batch.append(next_state)
      done_batch.append(done)
  
    return state_batch, action_batch, reward_batch, next_state_batch, done_batch

  def __len__(self):
    return len(self.buffer)

In [ ]:
class Critic(nn.Module):
  def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
    super(Critic, self).__init__()
    self.linear1 = nn.Linear(input_size, hidden_size1)     # dense 64 output (hidden_size1)
    self.linear2 = nn.Linear(hidden_size1, hidden_size2)   # dense 30 ouput  (hidden_size2)
    self.linear3 = nn.Linear(hidden_size2, output_size)    # dense 1 output  (output_size)

  def forward(self, state, action):
    """
    Params state and actions are torch tensors
    """
    x = torch.cat([state, action], 1)
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = self.linear3(x)          # difference between actor and critic

    return x

In [ ]:
class Actor(nn.Module):
  def __init__(self, input_size, hidden_size1 = 64, hidden_size2 = 64, output_size):
    super(Actor, self).__init__() #  lets you avoid referring to the base class explicitly
    # (size-of-each-input-sample, size-of-each-output-sample, bias =0 )
    self.linear1 = nn.Linear(input_size, hidden_size)    # dense 64 output 
    # Just do linear transformation, not classification
    self.linear2 = nn.Linear(hidden_size, hidden_size)   # dense 64 output
    self.linear3 = nn.Linear(hidden_size, output_size)   # dense action_dimension output
      
  def forward(self, state):
    """
    Param state is a torch tensor
    """
    x = F.relu(self.linear1(state))
    x = torch.tanh(self.linear2(x))
    x = torch.tanh(self.linear3(x))

    return x

In [ ]:
INIT = 0

#####################  hyper parameters  ####################
MAX_EP_STEPS = 100
LR_A = 0.002    # learning rate for actor
LR_C = 0.004    # learning rate for critic
GAMMA = 0.9     # reward discount
TAU = 0.01      # soft replacement
MEMORY_CAPACITY = 10000
BATCH_SIZE = 32

#####################  INIT  ####################
INIT = 0
#### INIT:
# RANDOM LOCATION WITH FADING K: 0
# TWO USER CASE                : 1
# SPECIAL CASE                 : 2
if INIT == 0:         # RANDOM LOCATION WITH FADING K #
  ct=500              # 
  MAX_EPISODES =10    #

elif INIT == 1:    # TWO USER CASE #
  Pn = 1              #
  K=2                 #
  MAX_EPISODES = 400  #
  MAX_EP_STEPS = 100  #
else:                 # SPECIAL CASE #
  Pn = 1
  K=10 # the number of grant based users
  MAX_EPISODES = 400

# env:
# hidden_size
# actor_learning_rate:   || critic_learning_rate
# gamma: discount factor
# tau: target network update parameters
# max_memory_size: 
class DDPG:
  def __init__(self, a_dim, s_dim, a_bound, hidden_size1=64, hidden_size2=30, 
               actor_learning_rate=LR_A, critic_learning_rate=LR_C, 
               gamma=GAMMA, tau=TAU, max_memory_size=MEMORY_CAPACITY):
    # Params
    self.num_states = s_dim
    self.num_actions = a_dim
    self.gamma = gamma
    self.tau = tau

    # Network layer definition
    a_hidden_layer_1_size = hidden_size1  
    a_hidden_layer_2_size = hidden_size1
    c_hidden_layer_1_size = hidden_size1
    c_hidden_layer_2_size = hidden_size2

    # Networks
    self.actor = Actor(self.num_states, a_hidden_layer_1_size,                             # input_size, hidden_size1
                       a_hidden_layer_2_size, self.num_actions)                            # hidden_size2, output_size
    self.actor_target = Actor(self.num_states, a_hidden_layer_1_size,                      # input_size, hidden_size1
                              a_hidden_layer_2_size, self.num_actions)                     # hidden_size2, output_size

    self.critic = Critic(self.num_states + self.num_actions, c_hidden_layer_1_size,        # input_size, hidden_size1
                         c_hidden_layer_2_size, self.num_actions)                          # hidden_size2, output_size
    self.critic_target = Critic(self.num_states + self.num_actions, c_hidden_layer_1_size, # input_size, hidden_size1
                         c_hidden_layer_2_size, self.num_actions)                          # hidden_size2, output_size

    # copy all parameters from actor network -> actor target network
    for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
      target_param.data.copy_(param.data)

    # copy all parameters from critic network -> critic target network
    for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
      target_param.data.copy_(param.data)
    
    # Training
    self.memory = Memory(max_memory_size)        
    self.critic_criterion  = nn.MSELoss()
    self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=actor_learning_rate)
    self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=critic_learning_rate)

  def get_action(self, state):
    state = Variable(torch.from_numpy(state).float().unsqueeze(0))
    action = self.actor.forward(state)
    action = action.detach().numpy()[0,0]
    return action
  
  def update(self, batch_size):
    states, actions, rewards, next_states, _ = self.memory.sample(batch_size)
    states = torch.FloatTensor(states)
    actions = torch.FloatTensor(actions)
    rewards = torch.FloatTensor(rewards)
    next_states = torch.FloatTensor(next_states)

    # Critic loss    
    # Critic depends on tuple[Q_target, mu_target, mu_main] 
    # mu_target = mu_main + delay + [noise from updating part of old mu]    
    # Q_target  = Q_main  + delay + [noise from updating part of old Q]
    Qvals = self.critic.forward(states, actions)
    next_actions = self.actor_target.forward(next_states)
    next_Q = self.critic_target.forward(next_states, next_actions.detach())
    Qprime = rewards + self.gamma * next_Q
    critic_loss = self.critic_criterion(Qvals, Qprime)
    
    
    # Actor loss
    # Simply the average sum of Q(s_i, mu(s_i))
    policy_loss = -self.critic.forward(states, self.actor.forward(states)).mean()
    
    # update networks
    self.actor_optimizer.zero_grad()
    policy_loss.backward()
    self.actor_optimizer.step()

    self.critic_optimizer.zero_grad()
    critic_loss.backward() 
    self.critic_optimizer.step()

    # update target networks 
    for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
      target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))
    
    for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
      target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))

In [ ]:
###############################  training  ####################################
s_dim = 3# dimsion of states
a_dim = 1# dimension of action
a_bound = 1 #bound of action
state_am = 10000


#GB_power_dBm = np.array([0,  10,  20,   30])
K_range = np.array([2,4,6,8, 10])
rate_DDPG = []
rate_greedy = []
rate_random = []
for k in range(len(K_range)):
  K = K_range[k]
  ratect_DDPG = 0
  ratect_greedy = 0
  ratect_random = 0
  for mct in range(ct):
    var = 1  # control exploration
    agent = DDPG(a_dim, s_dim, a_bound,                                  # necessary parameters
                hidden_size1=64, hidden_size2=30, 
                actor_learning_rate=LR_A, critic_learning_rate=LR_C, 
                gamma=GAMMA, tau=TAU, max_memory_size=MEMORY_CAPACITY)

    locationspace = np.linspace(1, 1000, num=K)
    location_vector = np.zeros((K, 2))
    location_vector[:, 1] = locationspace
    location_GF = np.array([[1,1]])# np.ones((1, 2))

    ##### fading for GB user
    hnx1 = np.random.randn(K, 2)
    hnx2 = np.random.randn(K, 2)
    fading_n = hnx1 ** 2 + hnx2 ** 2
    #### fading for GF user
    h0x1 = np.random.randn(1, 1)
    h0x2 = np.random.randn(1, 1)
    fading_0 =  h0x1[0,0] ** 2 + h0x2[0,0] ** 2

    Pn = 10**((30-30)/10)
    myenv = Env_cellular( MAX_EP_STEPS, s_dim, location_vector, location_GF, K,Pn, fading_n, fading_0)
    noise = OUNoise(env.action_space)

    ratek_DDPG = 0
    ratek_greedy = 0
    ratek_random = 0
    for i in range(MAX_EPISODES):
      batter_ini = myenv.reset()
      state = myenv.channel_sequence[i % myenv.K, :].tolist()  # the current GB user, 2 element [GB-GF, GB-BS]
      # s.append(myenv.h0)
      state.append(batter_ini)
      state = np.reshape(s, (1, s_dim))
      state = state * state_am  # amplify the state
      state_greedy = state
      state_random = state
      reward_dpgg_vector =[]
      reward_greedy_vector = []
      reward_random_vector = []
      #print(s[0,0:2])
      for step in range(MAX_EP_STEPS):

        # Add exploration noise
        a = agent.get_action(state)
        action = noise.get_action(action, t=step, )
        #print(myenv.location)
        reward, next_state, done = myenv.step(a, state / state_am, step)
        next_state = next_state * state_am
        #print(f"{step} state is {s}, action is {a}, reward is {r}, next state is {s_}")
        agent.memory.push(state, action, reward, new_state, done)
        if var>0.05:
          var *= .9998  # decay the action randomness
        if len(agent.memory) > BATCH_SIZE:
          agent.update(BATCH_SIZE)  
        state = next_state
        reward_dpgg_vector.append(r)

        ##### greedy
        reward_greedy, state_next_greedy, done = myenv.step_greedy(state_greedy/state_am, step)
        state_greedy = state_next_greedy*state_am
        reward_greedy_vector.append(reward_greedy)

        ##### random
        reward_random, state_next_random, done = myenv.step_random(state_random/state_am, step)
        state_random = state_next_random*state_am
        reward_random_vector.append(reward_random)

        if step == MAX_EP_STEPS-1:
          #print(f"Iteration., {k}--{mct},  Episode:, {i},  Reward: {sum(reward_dpgg_vector)/MAX_EP_STEPS},")
          #      f" Reward Greedy:   {sum(reward_greedy_vector)/MAX_EP_STEPS},"
          #      f" Reward random:   { sum(reward_random_vector)/MAX_EP_STEPS}, Explore: {var} ")
          break
        ratek_DDPG = sum(reward_dpgg_vector)/MAX_EP_STEPS
        ratek_greedy = sum(reward_greedy_vector)/MAX_EP_STEPS
        ratek_random = sum(reward_random_vector)/MAX_EP_STEPS

    tf.keras.backend.clear_session()
    ratect_DDPG +=ratek_DDPG
    ratect_greedy +=ratek_greedy
    ratect_random += ratek_random
    print(ratek_DDPG)
  print(f"Iteration., {k}--{mct},  Episode:, {i},  Reward: {ratect_DDPG/ct },")
  rate_DDPG.append(ratect_DDPG/ct )
  rate_greedy.append(ratect_greedy /ct )
  rate_random.append(ratect_random  /ct)

print(f"rate_greedy is {rate_greedy} and rate_random is {rate_random}")
#GB_power_label = ['0',  '10',  '20', '30']
GB_power_label = ['0', '5', '10', '15', '20', '25', '30']
K_rangex =  ['2', '4', '6', '8', '10']
plt.plot(K_rangex,rate_DDPG, "^-", label='DDPG: rewards')
plt.plot(K_rangex,rate_greedy, "+:", label='Greedy: rewards')
plt.plot(K_rangex,rate_random, "o--", label='Random: rewards')
plt.xlabel("The Number of Primary Users (K)")
plt.ylabel("  Average  Data Rate (NPCU)")
plt.legend()
plt.show()


''' Save final results'''
#np.savez_compressed('data/dataK', rate_DDPG=rate_DDPG, rate_greedy=rate_greedy, rate_random=rate_random)

# Code Test 

In [ ]:
MEMORY_CAPACITY = 10001
BATCH_SIZE = 32
pointer = 0
s_dim = 3
a_dim = 1

def store_transition(s, a, r, s_, pointer):
  rr = np.reshape(r,(1,1))
  aa = np.reshape(a,(1,1))
  ss = np.reshape(s,(1,1))
  ss_ = np.reshape(s_,(1,1))

  #print(f"state is {s}, action is {a}, reward is {r}, next state is {s_}")
  transition = np.hstack((ss, aa, rr, ss_))
  index = pointer % MEMORY_CAPACITY  # replace the old memory with new memory
  memory[index, :] = transition
  pointer += 1
  return pointer

memory = np.zeros((MEMORY_CAPACITY, 2 + 1 + 1), dtype=np.float32)
pointer = 0          # memory of Experience Replay used pointer

for k in range(10000):
  pointer = store_transition(k, k+1, k+2, k+3, pointer)
print ("memory is: \n{}\n".format(memory))

indices = np.random.choice(min(MEMORY_CAPACITY,pointer), size=BATCH_SIZE)
print ("all the indices are: \n{}\n".format(indices))
bt  = memory[indices, :]
print ("batch is: \n{}\n".format(bt))

# structure of observation is pre-defined
bs  = bt[:,       : s_dim]
ba  = bt[:,  s_dim: s_dim + a_dim]
br  = bt[:, -s_dim - 1: -s_dim]
bs_ = bt[:, -s_dim:]

print("bs is: \n{}\n".format(bs))
print("ba is: \n{}\n".format(ba))
print("br is: \n{}\n".format(br))
print("bs_ is: \n{}\n".format(bs_))

memory is: 
[[0.0000e+00 1.0000e+00 2.0000e+00 3.0000e+00]
 [1.0000e+00 2.0000e+00 3.0000e+00 4.0000e+00]
 [2.0000e+00 3.0000e+00 4.0000e+00 5.0000e+00]
 ...
 [9.9980e+03 9.9990e+03 1.0000e+04 1.0001e+04]
 [9.9990e+03 1.0000e+04 1.0001e+04 1.0002e+04]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]]

all the indices are: 
[7568 7011 2873 8428 9115 3907 5605 4596  900 2083 5473 5751 8237 5282
 9476 9935 5968 4306 1952 2803 2394 3701 4238 5539 1323 4488 8601 9796
 2011 8414 8638 5534]

batch is: 
[[7568. 7569. 7570. 7571.]
 [7011. 7012. 7013. 7014.]
 [2873. 2874. 2875. 2876.]
 [8428. 8429. 8430. 8431.]
 [9115. 9116. 9117. 9118.]
 [3907. 3908. 3909. 3910.]
 [5605. 5606. 5607. 5608.]
 [4596. 4597. 4598. 4599.]
 [ 900.  901.  902.  903.]
 [2083. 2084. 2085. 2086.]
 [5473. 5474. 5475. 5476.]
 [5751. 5752. 5753. 5754.]
 [8237. 8238. 8239. 8240.]
 [5282. 5283. 5284. 5285.]
 [9476. 9477. 9478. 9479.]
 [9935. 9936. 9937. 9938.]
 [5968. 5969. 5970. 5971.]
 [4306. 4307. 4308. 4309.]
 [1952. 1953. 1

In [ ]:
MAX_EP_STEPS = 100
K = 5

locationspace = np.linspace(1, 1000, num=K) 
# define a matrix of client-locations (each client includes 2 parts)
location_vector = np.zeros((K, 2))
# 2nd column definition
location_vector[:, 1] = locationspace
location_GF = np.array([[1,1]])# np.ones((1, 2))

print("locationspace:\n{}\n".format(locationspace))
print("location_vector:\n{}\n".format(location_vector))
print("location_GF:\n{}\n".format(location_GF))

locationspace:
[   1.    250.75  500.5   750.25 1000.  ]

location_vector:
[[   0.      1.  ]
 [   0.    250.75]
 [   0.    500.5 ]
 [   0.    750.25]
 [   0.   1000.  ]]

location_GF:
[[1 1]]



In [ ]:
##### fading for GB (grant-based) user
hnx1 = np.random.randn(K, 2)      # channel gain
hnx2 = np.random.randn(K, 2)      # channel gain
fading_n = hnx1 ** 2 + hnx2 ** 2
#### fading for GF (grant-free) user
h0x1 = np.random.randn(1, 1)      # channel gain 
h0x2 = np.random.randn(1, 1)      # channel gain
fading_0 =  h0x1[0,0] ** 2 + h0x2[0,0] ** 2

Pn = 10**((30-30)/10)

print("hnx1: \n{}\n".format(hnx1))
print("hnx2: \n{}\n".format(hnx2))
print("fading_n: \n{}\n".format(fading_n))
print("h0x1: \n{}\n".format(h0x1))
print("h0x2: \n{}\n".format(h0x2))
print("fading_0: \n{}\n".format(fading_0))
print("Pn: \n{}\n".format(Pn))

hnx1: 
[[-0.7025477  -0.76816182]
 [ 0.51816404  1.27695805]
 [ 1.36956635  0.2337245 ]
 [ 0.0316882  -0.81842374]
 [-1.1495627  -2.03515967]]

hnx2: 
[[-1.14285002  0.89788275]
 [-0.13430783 -0.06191845]
 [-0.38909785  1.52578619]
 [-1.52485374  1.78663988]
 [-1.97539743 -1.30185783]]

fading_n: 
[[1.79967945 1.39626602]
 [0.28653256 1.63445576]
 [2.02710913 2.38265063]
 [2.32618307 3.86189949]
 [5.2236894  5.8367087 ]]

h0x1: 
[[0.00397162]]

h0x2: 
[[0.05871606]]

fading_0: 
0.003463349416688293

Pn: 
1.0



In [ ]:
s_dim = 3 # dimsion of states
a_dim = 1 # dimension of action

################################################################################
# Noise calculator
BW = 10**6 # 10MHz
# -170 dBm/Hz: AWGN 
# -
sigma2_dbm = -170 + 10 * np.log10(BW) #  Thermal noise in dBm 
#sigma2_dbm = -94
noise = 10 ** ((sigma2_dbm - 30) / 10)
# self.Pn = self.Pn
################################################################################ 

#self.P0n = 10 #grant free user's power #transmit power of U0 at tn

#defines whatever you want =)))

# location_vector / location_GF: coordination of base station and secondary user

# separate distance between transmitter-receiver
# GF is reference coordinator
distance_GF = np.sqrt(np.sum((location_vector-location_GF)**2, axis=1))

# 
distance_GB = np.sqrt(np.sum((location_vector)**2, axis=1))


# path loss
# merge distance_GF and distance_GB into 1 matrix ( )
distance = np.matrix.transpose(np.array([distance_GF, distance_GB]))   # Kx2 matrix
distance = np.maximum(distance, np.ones((K,2))) # Less than one -> equal to one (normalize small value)
PL_alpha = 3
PL = 1/distance**PL_alpha/(10**3.17)

distance_GF0 = np.sqrt(np.sum( location_GF ** 2, axis=1))
distance0 = np.maximum(distance_GF0, 1)


PL0 = 1/(distance0 ** PL_alpha)/(10**3.17)

# channel = fading * pathloss ? 
hn = np.multiply( PL, fading_n)   
h0 = fading_0*PL0
#print(f"{hn} and {h0}")


channel_sequence = np.zeros((MAX_EP_STEPS,2))
for i in range(MAX_EP_STEPS):
    id_index = i % K
    # print("id_index: {} \n".format(id_index))
    channel_sequence[i,:] = hn[id_index,:]

print("distance_GF\n{}\n".format(distance_GF))
print("distance_GF0\n{}\n".format(distance_GF0))
print("distance_GB\n{}\n".format(distance_GB))
print("distance\n{}\n".format(distance))
print("distance0\n{}\n".format(distance0))
print("PL0\n{}\n".format(PL0))                         
print("h0\n{}\n".format(h0))                           
                                                       
print("channel_sequence\n{}\n".format(channel_sequence))


distance_GF
[  1.         249.75200199 499.501001   749.25066733 999.0005005 ]

distance_GF0
[1.41421356]

distance_GB
[   1.    250.75  500.5   750.25 1000.  ]

distance
[[   1.            1.        ]
 [ 249.75200199  250.75      ]
 [ 499.501001    500.5       ]
 [ 749.25066733  750.25      ]
 [ 999.0005005  1000.        ]]

distance0
[1.41421356]

PL0
[0.00023903]

h0
[8.27849358e-07]

channel_sequence
[[1.21673264e-03 9.43991683e-04]
 [1.24350361e-11 7.00890777e-11]
 [1.09968434e-11 1.28483726e-11]
 [3.73906036e-12 6.18276805e-12]
 [3.54225831e-12 3.94609938e-12]
 [1.21673264e-03 9.43991683e-04]
 [1.24350361e-11 7.00890777e-11]
 [1.09968434e-11 1.28483726e-11]
 [3.73906036e-12 6.18276805e-12]
 [3.54225831e-12 3.94609938e-12]
 [1.21673264e-03 9.43991683e-04]
 [1.24350361e-11 7.00890777e-11]
 [1.09968434e-11 1.28483726e-11]
 [3.73906036e-12 6.18276805e-12]
 [3.54225831e-12 3.94609938e-12]
 [1.21673264e-03 9.43991683e-04]
 [1.24350361e-11 7.00890777e-11]
 [1.09968434e-11 1.28483726e-11